In [1]:
from __future__ import print_function

import collections
import time

import import_ipynb
from config import *

# Import Python wrapper for or-tools CP-SAT solver.
from ortools.sat.python import cp_model

from IPython.core.debugger import set_trace

importing Jupyter notebook from config.ipynb


In [2]:

def placement_ortools(config, NB_JOBS, NB_MACHINES, MACHINES, DURATION):

    """Jobshop problem."""
    
    # Create the model.
    model = cp_model.CpModel()

    jobs_data = [[(xx,yy) for xx,yy in zip(x,y)] for x, y in zip(MACHINES, DURATION)]
    
    """
    jobs_data = [  # task = (machine_id, processing_time).
        [(0, 3), (1, 2), (2, 2)],  # Job0
        [(0, 2), (2, 1), (1, 4)],  # Job1
        [(1, 4), (2, 3)]  # Job2
    ]
    """
    machines_count = 1 + max(task[0] for job in jobs_data for task in job)
    all_machines = range(machines_count)

    # Computes horizon dynamically as the sum of all durations.
    horizon = sum(task[1] for job in jobs_data for task in job)

    # Named tuple to store information about created variables.
    task_type = collections.namedtuple('task_type', 'start end interval')
    # Named tuple to manipulate solution information.
    assigned_task_type = collections.namedtuple('assigned_task_type',
                                                'start job index duration')

    # Creates job intervals and add to the corresponding machine lists.
    all_tasks = {}
    machine_to_intervals = collections.defaultdict(list)

    for job_id, job in enumerate(jobs_data):
        for task_id, task in enumerate(job):
            machine = task[0]
            duration = task[1]
            suffix = '_%i_%i' % (job_id, task_id)
            start_var = model.NewIntVar(0, horizon, 'start' + suffix)
            end_var = model.NewIntVar(0, horizon, 'end' + suffix)
            interval_var = model.NewIntervalVar(start_var, duration, end_var,
                                                'interval' + suffix)
            all_tasks[job_id, task_id] = task_type(
                start=start_var, end=end_var, interval=interval_var)
            machine_to_intervals[machine].append(interval_var)

    # Create and add disjunctive constraints.
    for machine in all_machines:
        model.AddNoOverlap(machine_to_intervals[machine])

    # Precedences inside a job.
    for job_id, job in enumerate(jobs_data):
        for task_id in range(len(job) - 1):
            model.Add(all_tasks[job_id, task_id +
                                1].start >= all_tasks[job_id, task_id].end)

    # Makespan objective.
    obj_var = model.NewIntVar(0, horizon, 'makespan')
    model.AddMaxEquality(obj_var, [
        all_tasks[job_id, len(job) - 1].end
        for job_id, job in enumerate(jobs_data)
    ])
    model.Minimize(obj_var)

    # Solve model.
    solver = cp_model.CpSolver()
    solver.parameters.max_time_in_seconds = config.timeout
    status = solver.Solve(model)
    
    placements = [[] for m in range(NB_MACHINES)]

    if status == cp_model.OPTIMAL:
        # Create one list of assigned tasks per machine.
        assigned_jobs = collections.defaultdict(list)
        for job_id, job in enumerate(jobs_data):
            for task_id, task in enumerate(job):
                machine = task[0]
                assigned_jobs[machine].append(
                    assigned_task_type(
                        start=solver.Value(all_tasks[job_id, task_id].start),
                        job=job_id,
                        index=task_id,
                        duration=task[1]))

        # Create per machine output lines.

        for machine in all_machines:
            # Sort by starting time.
            assigned_jobs[machine].sort()

            for assigned_task in assigned_jobs[machine]:
                
                placements[machine].append([assigned_task.job, assigned_task.index])
                
                
        # Finally print the solution found.
        #print('Optimal Schedule Length: %i' % solver.ObjectiveValue())

    return placements, solver.ObjectiveValue()


In [3]:
if __name__ == "__main__":
    
    
    config = Config()    
    config.machine_profile = "xsmall_default"
    config.job_profile = "xsmall_default"
    config.reconfigure()
    
   
    
    # Read the input data file.
    # Available files are jobshop_ft06, jobshop_ft10 and jobshop_ft20
    # First line contains the number of jobs, and the number of machines.
    # The rest of the file consists of one line per job.
    # Each line contains list of operations, each one given by 2 numbers: machine and duration
   
    filename = "datasets/inference/dataset_xsmall.data"
    
    with open(filename, "r") as file:
        NB_JOBS, NB_MACHINES = [int(v) for v in file.readline().split()]
        JOBS = [[int(v) for v in file.readline().split()] for i in range(NB_JOBS)]


    #-----------------------------------------------------------------------------
    # Prepare the data for modeling
    #-----------------------------------------------------------------------------

    # Build list of machines. MACHINES[j][s] = id of the machine for the operation s of the job j
    MACHINES = [[JOBS[j][2 * s] for s in range(NB_MACHINES)] for j in range(NB_JOBS)]

    # Build list of durations. DURATION[j][s] = duration of the operation s of the job j
    DURATION = [[JOBS[j][2 * s + 1] for s in range(NB_MACHINES)] for j in range(NB_JOBS)]

    start = time.time()
    placements, makespan = placement_ortools(config, NB_JOBS, NB_MACHINES, MACHINES, DURATION)
    end = time.time()
    print("time: ", end - start)
    print("makespan: ", makespan)

time:  0.004598855972290039
makespan:  24.0


In [4]:
#print(placements)